## **NoneとかNullとかNaNとか調べてみた**

いつでも迷う(混乱する)pythonのまとめ

In [2]:
import io
import math
import numpy as np
import pandas as pd

|設定|None|np.nan<br>math.nan|-|
|判定|xx is None|np.math<br> xx is np.nan<br> np.isnan(xx)<br> math.isnan(xx)<br>math.nan<br> xx is math.nan<br> np.isnan(xx)<br> math.isnan(xx)||

pythonではオブジェクトが値を持たない場合にNoneとかnullとかnanがあり、いつも迷うので以下にまとめ。  
<br>
ざっくり言うと、  

|値|説明|
|--|--|
|None|表示上はNone、値がないということを表す. 'NoneType'という型のシングルトンオブジェクト|
|nan|表示上はnanまたはNaN、値がないということを表す. 型はfloat.  実はnp.nanとmath.nanがある|
|null|nullという値、型は存在しない.pandasにisnull()というメッソドがあるため混乱を招いている.<br>後述動作から推測すると、Noneとnanを合わせた概念か.|

In [56]:
print(None, type(None))
print(np.nan, type(np.nan))
print(math.nan, type(math.nan))
#print(null, type(null))     #==> Error

None <class 'NoneType'>
nan <class 'float'>
nan <class 'float'>


### **単独での判定**

||xx is None|np.isnan(xx)|math.isnan(xx)|xx == np.isnan|xx == math.isnan|
|--|--|--|--|--|--|
|None|**True**|TypeError|TypeError|**False**|**False**|
|np.nan|**False**|**True**|**True**|**False**|**False**|
|math.nan|**False**|**True**|**True**|**False**|**False**|


In [86]:
xx = None
print('\n## xx =', xx)
print('xx is None, xx=',       xx is None)
# print('np.isnan(xx), xx=',     np.isnan(xx))   #==> TypeError
# print('math.isnan(xx), xx=',   math.isnan(xx)) #==> TypeError
print('xx == np.isnan, xx=',   xx == np.isnan)
print('xx == math.isnan, xx=', xx == math.isnan)

xx = np.nan
print('\n## xx =', xx)
print('xx is None, xx=',       xx is None)
print('np.isnan(xx), xx=',     np.isnan(xx))
print('math.isnan(xx), xx=',   math.isnan(xx))
print('xx == np.isnan, xx=',   xx == np.isnan)
print('xx == math.isnan, xx=', xx == math.isnan)

xx = math.nan
print('\n## xx =', xx)
print('xx is None, xx=',       xx is None)
print('np.isnan(xx), xx=',     np.isnan(xx))
print('math.isnan(xx), xx=',   math.isnan(xx))
print('xx == np.isnan, xx=',   xx == np.isnan)
print('xx == math.isnan, xx=', xx == math.isnan)


## xx = None
xx is None, xx= True
xx == np.isnan, xx= False
xx == math.isnan, xx= False

## xx = nan
xx is None, xx= False
np.isnan(xx), xx= True
math.isnan(xx), xx= True
xx == np.isnan, xx= False
xx == math.isnan, xx= False

## xx = nan
xx is None, xx= False
np.isnan(xx), xx= True
math.isnan(xx), xx= True
xx == np.isnan, xx= False
xx == math.isnan, xx= False


<br>

### **numpyの場合**

np.nan, math.nanを含むnumpy array

In [127]:
arr1 = np.array([1, np.nan, math.nan])
print(arr1, type(arr1), '\n', type(arr1[1]), type(arr1[2]))
print(np.isnan(arr1))
# print(arr1.isnan()) # ==> AttributeError: 'numpy.ndarray' object has no attribute 'isnan'

[ 1. nan nan] <class 'numpy.ndarray'> 
 <class 'numpy.float64'> <class 'numpy.float64'>
[False  True  True]


nummpy array内にnp.nanとmath.nanが混在していても、isnan関数は区別なく動作する.  
なお、メソッドは無いようだ.

<br>

Noneを含んだ場合のnumpy array

In [142]:
arr2 = np.array([1, None, np.nan, math.nan]) # type混在
print(arr2, type(arr2), '\n', type(arr2[0]), type(arr2[1]), type(arr2[2]))
# print(np.isnan(arr2))                      # ==> TypeError

[1 None nan nan] <class 'numpy.ndarray'> 
 <class 'int'> <class 'NoneType'> <class 'float'>


Noneを含んだarrayは作成できるが、そのままでisnan関数適用するとTypeErrorになる.

In [143]:
arr3 = arr2.astype(float)
print(arr3, type(arr3), '\n', type(arr3[0]), type(arr3[1]), type(arr3[2]))
print(np.isnan(arr3))

[ 1. nan nan nan] <class 'numpy.ndarray'> 
 <class 'numpy.float64'> <class 'numpy.float64'> <class 'numpy.float64'>
[False  True  True  True]


Noneを含んだarrayをfloatに変換すると、isnan関数が適用できるようになる.
その時、Noneはnp.nanに変換される

<br>

### **pandas.Seriesの場合**

In [145]:
arr2 = np.array([1, None, np.nan, math.nan]) # type混在
sr1 = pd.Series(arr2)
print(sr1, type(sr), '\n', type(sr1[0]), type(sr1[1]), type(sr1[2]))
print(sr1.isnull())

0       1
1    None
2     NaN
3     NaN
dtype: object <class 'pandas.core.series.Series'> 
 <class 'int'> <class 'NoneType'> <class 'float'>
0    False
1     True
2     True
3     True
dtype: bool


型が混在しているnumpy.arrからpandas.Seriseに変換した場合は、Serise内でも型が混在しているが、is.nullメソッドはNone, np.nan, math.nanを期待通りに処理してくれる.

In [257]:
sr2 = pd.Series([1, None, np.nan, math.nan])  # type混在
print(sr2, type(sr2), '\n', type(sr2[0]), type(sr2[1]), type(sr2[2]))
print(sr2.isnull())
sr3 = pd.Series(['a', None, np.nan, math.nan])  # type混在
print(sr3, type(sr3), '\n', type(sr3[0]), type(sr3[1]), type(sr3[2]))
print(sr3.isnull())

0    1.0
1    NaN
2    NaN
3    NaN
dtype: float64 <class 'pandas.core.series.Series'> 
 <class 'numpy.float64'> <class 'numpy.float64'> <class 'numpy.float64'>
0    False
1     True
2     True
3     True
dtype: bool
0       a
1    None
2     NaN
3     NaN
dtype: object <class 'pandas.core.series.Series'> 
 <class 'str'> <class 'NoneType'> <class 'float'>
0    False
1     True
2     True
3     True
dtype: bool


型が混在しているリストからpandas.Seriseに変換した場合では、None, nan以外が数値の場合には、Serise全体がnumpy.floatになり、Noneはnanに変換される。None, nan以外にstr(文字(列))がある場合には型が混在するSeriseになる.いずれの場合もis.nullメソッドはNone, np.nan, math.nanを期待通りに処理してくれる.

<br>

### **pandas.read_csvの場合**

In [259]:
csv_text =                    \
',A,B,C,D,E,F\n'             +\
'0,a,b,1,2,3,4\n'            +\
'1,,,,,,\n'                  +\
'2,None,,None,,"",2\n'       +\
'3,NaN,NaN,NaN,NaN,NaN,NaN\n'+\
'4,nan,nan,nan,nan,nan,nan'
df = pd.read_csv(io.StringIO(csv_text), header=0, index_col=0)
display(df, df.dtypes)

df_type = df.copy()
for c in df_type.columns:
    df_type[c] = [str(type(df.loc[i,c])) for i in df.index]
    df_type[c] = df_type[c].str.replace('class ','')
display(df_type)

,A,B,C,D,E,F
0,a,b,1,2.0,3.0,4.0
1,NaN,NaN,NaN,NaN,NaN,NaN
2,None,NaN,None,NaN,NaN,2.0
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN


A     object
B     object
C     object
D    float64
E    float64
F    float64
dtype: object

None


,A,B,C,D,E,F
0,<'str'>,<'str'>,<'str'>,<'numpy.float64'>,<'numpy.float64'>,<'numpy.float64'>
1,<'float'>,<'float'>,<'float'>,<'numpy.float64'>,<'numpy.float64'>,<'numpy.float64'>
2,<'str'>,<'float'>,<'str'>,<'numpy.float64'>,<'numpy.float64'>,<'numpy.float64'>
3,<'float'>,<'float'>,<'float'>,<'numpy.float64'>,<'numpy.float64'>,<'numpy.float64'>
4,<'float'>,<'float'>,<'float'>,<'numpy.float64'>,<'numpy.float64'>,<'numpy.float64'>


pandas.read_csvでファイルまたはテキストからpandas.DataFrameを作成した場合、
- Noneは文字列"None"として読み込まれる
- CSV上で値を持たない場合および長さが0の文字列("")を読み込むと、nanとして読み込まれる

以降は各列に対して、pandas.Seriesをリストから作成した場合に準じた動作となる。

<br>

### **pandas/numpy メッソド/関数**  

pdxx = pd.Series([1, 2, 3, None, np.nan, math.nan])  
npxx = np.array( [1, 2, 3, np.nan, np.nan, math.nan])  

||pd.Series|np.array|Notes|
|--|--|--|--|
|**pdxx.sum()**            |**✓**|-||
|**pdxx.sum(skipna=True)** |**✓**|-||
|**pdxx.sum(skipna=False)**|**nan**|-||
|**np.sum(xx)**            |**✓**|**nan**||
|**np.nansum(xx)**         |**✓**|**✓**||
|**npxx.sum()**            |-|**nan**||
|**npxx.nansum()**         |-|-|メソッドは無い|

pandasのメッソドでは、nanは除かれて計算される(デフォルト:skipna=True)  
numpyのメッソド/関数では、nanが含まれているとnanを返す  
numpyでnanを除いて計算するには、numpy関数「nan元の関数名」を使用する. ただしメソッドは無い  


In [24]:
sr8 = pd.Series([1, 2, 3, None, np.nan, math.nan])
print(sr8)
print(sr8.sum()) # 6.0
print(sr8.sum(skipna=True)) # 6.6
print(sr8.sum(skipna=False)) # nan
print(np.sum(sr8)) # 6.0

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
5    NaN
dtype: float64
6.0
6.0
nan
6.0


In [28]:
arr8 = np.array([1, 2, 3, np.nan, np.nan, math.nan])
print(arr8)
print(np.sum(sr8),     np.min(sr8),     np.mean(sr8)    ) # 6.0
print(np.sum(arr8),    np.min(arr8),    np.mean(arr8)   ) # nan
print(np.nansum(sr8),  np.nanmin(sr8),  np.nanmean(sr8) ) # 6.0
print(np.nansum(arr8), np.nanmin(arr8), np.nanmean(arr8)) # 6.0
print(arr8.sum(),      arr8.min(),      arr8.mean()     ) # nan
# print(arr8.nansum(),   arr8.nansum(),   arr8.nansum()) #=> ERROR

[ 1.  2.  3. nan nan nan]
6.0 1.0 2.0
nan nan nan
6.0 1.0 2.0
6.0 1.0 2.0
nan nan nan
